<!--NAVIGATION-->
< [Delineate streams](Streams.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Automate stream and catchment delineation](AutomateDelineation.ipynb) >

# 7. Catchment Delineation
In this section we're going to delineate the catchment of an outlet and all catchments in the DEM.

Let's first import the libraries and set the path to the data.

In [ ]:
import os
from pcraster import *
os.chdir('./Data/')

## 7.1 Delineate the catchment of a specific outlet

First we need to identify an outlet on the delineated stream of the previous step. You can use the result from the Strahler order method or the flow accumulation method for finding the pixels that are part of the river. You can use QGIS or Aguila to find the coordinate. In Aguila you can do that using the crosshair tool.

You can use the fields below to load the river map from the previous section, visualise with Aguila and find the coordinates of a pixel on the river map that we want to use as outlet for the catchment that we're going to delineate.

In [ ]:
river = readmap("stream8.map")

Visualise the river map

Let's use 288880.648,5675880.258 as coordinates of the outlet.
PCRaster comes with a tool <code>col2map</code>, which reads a textfile in the format:

`x y id`

and converts it to a PCRaster map. You can run the tool from the commandline, but here we're going to wrap it into a Python function. More info about the `col2map` tool can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/app_col2map.html)

The tool has arguments for the data type and needs a clone map. In our case the outlet will be a nominal raster with one pixel that has an id number. For the clone we can use the river map.

In [ ]:
def col2map(x,y,id,datatype,clone):
    with open('location.txt', 'w') as f:
        f.write(str(x) + ' ' + str(y) + ' ' + str(id))
    cmd = 'col2map -{0} location.txt location.map --clone {1}'.format(datatype,clone)
    print(cmd)
    os.system(cmd)
    Map = readmap('location.map')
    return Map

x = 288880.648
y = 5675880.258
id = 1
datatype = 'N'
clone = 'stream8.map'
Outlet = col2map(x,y,id,datatype,clone)
aguila(Outlet,river)

The function creates a text file. We wrote a string with "x y id" to the file. Then the command was constructed as a string and we run the command string with <code>os.system</code>.

We have also printed the command string after running the script. This command could be typed at the command prompt.

Now we have the outlet defined in our delineated river, we can delineate the catchment that contributes to that outlet using the <code>catchment</code> operation. The <code>catchment</code> operation needs the flow direction map and the outlet map as input. More information about the <code>catchment</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_catchment.html).

In [ ]:
FlowDirection = readmap('flowdir.map')
RurCatchment = catchment(FlowDirection,Outlet)

Visualise the result with Aguila.

## 7.2 Delineate all catchments in the DEM

That's the catchment of a specific outlet. If we want to automatically derive all catchments in the raster we need to find the pits first, which can be considered as outlets. This can be done with the <code>pit</code> operation. More information about the <code>pit</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_pit.html). It only needs the flow direction map as input.

In [ ]:
outlets = pit(FlowDirection)

Visualise the result with Aguila.

*Where are the pits located?*

*How many are there?*

Now we can use those outlets in the <code>catchment</code> operation to derive all catchments in the raster.

In [ ]:
Catchments = catchment(FlowDirection, outlets)

Visualise the result with Aguila.

*What do you see?*

*Why are there so many catchments at the boundary of the area?*

In the next section we're going to combine all code into one script to automate the process as much as possible.

<!--NAVIGATION-->
< [Delineate streams](Streams.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Automate stream and catchment delineation](AutomateDelineation.ipynb) >